In [4]:
import os
import asyncio
import pandas as pd
import pandas_datareader.data as pdr

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

import itertools
import pmdarima as pm
import time
import warnings
warnings.filterwarnings('ignore')

In [5]:
from datetime import datetime, timedelta

# 현재 날짜 가져오기
today = datetime.now() 

# 날짜를 YYYYMMDD 형식으로 변환
today_str = today.strftime('%Y%m%d')
one_year_ago = today - timedelta(days=365)
one_year_ago_str = one_year_ago.strftime('%Y%m%d')
print(today_str,one_year_ago_str)

20240618 20230619


In [6]:
def fetch_prices(cursor, code):
    data = []
    query = f'select date, price from stocks_products_details where date between "2024-01-01" and "2024-06-20" and stock_code = "{code}";'
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        data.append(row)
    df = pd.DataFrame(data, columns=['date', 'price'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

In [24]:
from mysqlconnect import connect_to_mysql

connection = connect_to_mysql()

cursor = connection.cursor()
# returned = ARIMA("000080")        
# print(f"returned : {returned}")

hitit-user
Connected to MySQL database


In [25]:
# async def insert_data() :
#     query = "select * from stocks_products"
#     cursor.execute(query)
#     row = cursor.fetchone()
#     while row is not None:
#         code = row[0]
#         print(f"code : {code}")
#         returned = await get_arima_predict(code)
#         print(returned)
#         # time.sleep(5)
#         row = cursor.fetchone()
# insert_data()


def return_insert_query(percent, price, code):
    return f"""
UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = {percent}, 
predict_price = {price}
WHERE stock_code = {code};
"""

query = "select * from stocks_products"
cursor.execute(query)
rows = cursor.fetchall()  # 모든 결과를 한 번에 가져옴

for row in rows:
    code = row[0]
    print(f"code : {code}")
    # {'예측값': 20243.78831164702, '예측일': '20240711', '등락률': -2.44}
    returned = get_arima_predict(code)
    print(returned)
    if 'error' not in returned:
        query = return_insert_query(returned['등락률'], round(returned['예측값'], 2), code)
        print(query)
    else:
        query = return_insert_query(-100, -100, code)
        print(query)
        
    cursor.execute(query)
    connection.commit()  # 변경 사항을 커밋하여 데이터베이스에 반영


code : 000020
hitit-user
Connected to MySQL database
{'error': 'error'}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = -100, 
predict_price = -100
WHERE stock_code = 000020;

code : 000040
hitit-user
Connected to MySQL database
{'error': 'error'}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = -100, 
predict_price = -100
WHERE stock_code = 000040;

code : 000050
hitit-user
Connected to MySQL database
{'error': 'error'}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = -100, 
predict_price = -100
WHERE stock_code = 000050;

code : 000070
hitit-user
Connected to MySQL database
{'error': 'error'}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = -100, 
predict_price = -100
WHERE stock_code = 000070;

code : 000080
hitit-user


Connected to MySQL database
{'예측값': 20243.78831164702, '예측일': '20240711', '등락률': -2.44}
{'예측값': 20243.78831164702, '예측일': '20240711', '등락률': -2.44}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = -2.44, 
predict_price = 20243.79
WHERE stock_code = 000080;

code : 000100
hitit-user
Connected to MySQL database
{'예측값': 79926.58706602725, '예측일': '20240711', '등락률': 7.57}
{'예측값': 79926.58706602725, '예측일': '20240711', '등락률': 7.57}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = 7.57, 
predict_price = 79926.59
WHERE stock_code = 000100;

code : 000120
hitit-user
Connected to MySQL database
{'예측값': 88927.77777777796, '예측일': '20240711', '등락률': -8.7}
{'예측값': 88927.77777777796, '예측일': '20240711', '등락률': -8.7}

UPDATE stocks_products 
SET pred_update = now(), 
predict_precent = -8.7, 
predict_price = 88927.78
WHERE stock_code = 000120;

code : 000140
hitit-user
Connected to MySQL database
{'error': 'error'}

UPDATE stocks_products 
SET pred_update = now(), 

In [9]:
def plot(current,predict):
    plt.plot(current, label='종가')
    plt.plot(predict, label='Test')
    plt.legend()
    plt.show()

def get_arima_predict(code):
    try :
        connection = connect_to_mysql()
        cursor = connection.cursor()
        prices = fetch_prices(cursor,code)

        model = auto_arima(prices, start_p=1, start_q=1, 
                                    max_p=3, max_1=3, m=3, seasonal=False, # sarima(seasonal=True)
                                    d=1, D=0,
                                    max_P=3, max_Q=3, 
                                    # trace=True,
                                    error_action='ignore',
                                    suppress_warnings=True, 
                                    stepwise=False)
        pred = model.predict(n_periods=30).to_list()

        date_range = pd.date_range(start=prices.index[-1], periods=30, freq='D')
        future_data = pd.DataFrame(data=pred, index=date_range, columns=['Value'])
        result = {'예측값' : future_data.iloc[-1].item(), '예측일' : future_data.index[-1].strftime('%Y%m%d') ,"등락률" : round( (future_data.iloc[-1].item()/prices.iloc[-1].item() -1 )*100,2)}
        print(result)
        return result
    except Exception as e:
        return {"error" : "error"}

In [85]:
from transformers import pipeline
from flask import Flask, request, jsonify
import numpy as np
import joblib

# 감정 분석 파이프라인 로드
mapping = {3: '매우공격적', 2: '공격적', 1: '안정적', 0: '소극적'}
map_func = np.vectorize(mapping.get)

app = Flask(__name__)

@app.route('/arima/<code>', methods=['POST'])
def predict(code):
    try:
        arima_result = get_arima_predict(code)
        return jsonify(arima_result)
        
    except Exception as e:
        return jsonify({'error': str(e)})
    
if __name__ == "__main__":
    from waitress import serve
    serve(app, host="0.0.0.0", port=8081)

OSError: [Errno 98] Address already in use